In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_excel("NZ_Admin_JOBS.xlsx")
pd.set_option('display.max_colwidth', None)

In [3]:
# Rename columns
new_column_names = ["position", "url", "company", "location", "elapsed_time_since_posted", "classification"]
df.columns = new_column_names

In [4]:
# Position Capitalised
df["position"] = df["position"].str.title()

In [5]:
df.shape

(2708, 6)

In [6]:
# Check unique URLs
df["url"].nunique()

2708

In [7]:
def extract_and_clean(row):
    # Function to deduplicate by splitting the string in the middle
    def deduplicate(s):
        middle = len(s) // 2
        return s[:middle].strip()

    # Split the string at the comma, if present
    parts = row.split(',', 1)
    location_area_part = parts[0]
    benefit_part = parts[1].strip() if len(parts) > 1 else pd.NA

    # Check if "area:" is in the location_area_part
    if "area:" in location_area_part:
        # Find the index where "area:" starts
        area_index = location_area_part.find("area:")
        
        # Split the string into location and area parts
        location_part = location_area_part[:area_index]
        area_part = location_area_part[area_index:]
    else:
        # If "area:" is not present, the entire string is treated as the location part
        location_part = location_area_part
        area_part = ""

    # Deduplicate and clean location and area
    location = deduplicate(location_part.replace("location:", "").strip())
    area = deduplicate(area_part.replace("area:", "").strip()) if area_part else pd.NA

    return pd.Series([location, area, benefit_part])

# Assuming df is your DataFrame with a column named 'location'
df[['location', 'area', 'benefit']] = df['location'].apply(extract_and_clean)


In [8]:
def extract_classification(row):
    # Function to deduplicate by splitting the string in the middle
    def deduplicate(s):
        middle = len(s) // 2
        return s[:middle].strip()

    # Check if "classification:" is in the string
    if "classification:" in row:
        # Split the string into classification and subClassification parts
        parts = row.split("subClassification:")
        classification_part = parts[0]
        subClassification_part = parts[1] if len(parts) > 1 else pd.NA
    else:
        # If "classification:" is not present, set both parts to NA
        return pd.Series([pd.NA, pd.NA])

    # Deduplicate and clean classification and subClassification
    classification = deduplicate(classification_part.replace("classification:", "").strip())
    subClassification = deduplicate(subClassification_part.replace("subClassification:", "").strip()) if subClassification_part else pd.NA

    return pd.Series([classification, subClassification])

# Assuming df is your DataFrame with a column named 'classification'
df[['classification', 'sub_classification']] = df['classification'].apply(extract_classification)

In [9]:
# Function to check and replace NaN in 'company' with 'Private Advertiser'
def replace_nan_with_private_advertiser(row):
    if 'Private Advertiser' in row['elapsed_time_since_posted'] and pd.isna(row['company']):
        return 'Private Advertiser'
    else:
        return row['company']

# Apply the function to each row
df['company'] = df.apply(replace_nan_with_private_advertiser, axis=1)

In [10]:
def convert_to_hours(time_str):
    # Using regex to extract the time value and unit
    match = re.search(r'(\d+)([dhm])', time_str)
    if match:
        value, unit = int(match.group(1)), match.group(2)
        if unit == 'd':
            return value * 24  # days to hours
        elif unit == 'h':
            return value      # already in hours
        elif unit == 'm':
            return value * 24 * 30  # months to hours
    return pd.NA  # default case if no match

# Apply the conversion function to each row
df['elapsed_time_since_posted'] = df['elapsed_time_since_posted'].apply(convert_to_hours)

In [11]:
print(df["benefit"].unique())

[<NA> '$20 per hour' '$20 - $24.99 per hour' '$19 to $22 per hour'
 '$40,000 - $49,999' '$25 per hour' 'Includes life and health insurance'
 '$60 000 - $70 000 per annum' '$20.00 per hour' '$55,000 - $69,999'
 '$20.00 per hour.' '$21 per hour' '$55,000 - $64,999'
 '$50,000 - $55,000 plus benefits' 'incl Life & Health Insurance'
 '$45,000 - $49,999' '$55,000 - $60,000 per annum pro rata'
 '$48,835 - $63,221' '$20 - $29.99 per hour' '$24-$28' 'Up to $27ph'
 '8% holiday pay' '$45,000 - $59,999' 'NZD25 - NZD28 per hour'
 'Competitive Salary' '$22 per hour' 'NZD75000 per annum'
 '$75k - $85k p.a.' '$24 - $25 + 8% holiday pay!' '$50,000 per annum.'
 'bonus plus medical insurance' '$20 - $25 per hour' 'Good salary'
 '$30 - $38 p.h. + 8% Holiday Pay' '$45-$55,000' '$50,000 - $54,999'
 'Up to $23 p.h. + + 8% Holiday Pay' 'Up to $22.00 p.h.'
 'NZD69,726-94,127p.a.+Retirement contributions'
 '$60k-$70k depending on experience!' '$22.00 - $23.00 phr'
 '$27 - $33 p.h. + 8% holiday pay' '$60K - $80K

In [12]:
# def standardize_benefits(benefit):
#     if pd.isna(benefit) or benefit == '<NA>':
#         return 'Undefined'

#     # Adjust regex to capture numerical values and remove commas and spaces
#     matches = re.findall(r'(\d{1,3}(?:[,\s]\d{3})*\.?\d*)(k)?', benefit)
#     if not matches:
#         return 'Undefined'

#     numbers = [float(num.replace(',', '').replace(' ', '')) * 1000 if k else float(num.replace(',', '').replace(' ', '')) for num, k in matches]

#     # Determine if the benefit is an hourly or annual salary
#     is_annual_salary = 'per annum' in benefit or 'p.a.' in benefit or any(k for _, k in matches)
#     is_hourly_salary = 'per hour' in benefit or 'p.h.' in benefit or 'phr' in benefit or (not is_annual_salary and '-' in benefit and all(num < 1000 for num in numbers))

#     if is_hourly_salary:
#         # For hourly salary, directly return the average or single value
#         return sum(numbers) / len(numbers)

#     elif is_annual_salary:
#         # For annual salary, convert to hourly rate
#         annual_salary = sum(numbers) / len(numbers)
#         return annual_salary / 2080

#     elif any(word in benefit for word in ['insurance', 'holiday pay', 'bonus', 'benefits', 'Super', 'kiwisaver']):
#         return 'Non-Monetary Benefit'

#     return 'Undefined'

In [13]:
# def standardize_benefits(benefit):
#     if pd.isna(benefit) or benefit == '<NA>':
#         return 'Undefined'

#     # Adjust regex to capture numerical values and remove commas and spaces
#     # Handles numbers with commas, spaces, and 'k' as thousand indicators
#     matches = re.findall(r'(\d{1,3}(?:[,\s]\d{3})*\.?\d*)', benefit.replace('k', '000'))
#     if not matches:
#         return 'Undefined'

#     numbers = [float(match.replace(',', '').replace(' ', '')) for match in matches]

#     if 'per hour' in benefit or 'p.h.' in benefit or 'phr' in benefit:
#         if len(numbers) == 2:
#             return sum(numbers) / 2  # Average of a range for hourly rate
#         return numbers[0]  # Single value for hourly rate

#     elif 'per annum' in benefit or 'p.a.' in benefit or '-' in benefit:
#         if len(numbers) == 2:
#             # Average of a range for annual salary, then convert to hourly rate
#             return sum(numbers) / 2 / 2080
#         return numbers[0] / 2080  # Single annual salary value to hourly rate

#     elif any(word in benefit for word in ['insurance', 'holiday pay', 'bonus', 'benefits', 'Super', 'kiwisaver']):
#         return 'Non-Monetary Benefit'

#     return 'Undefined'

# # Apply the function
# df['benefit'] = df['benefit'].apply(standardize_benefits)

In [14]:
def separate_salaries(benefit):
    if pd.isna(benefit) or benefit == '<NA>' or not any(char.isdigit() for char in benefit):
        return pd.NA, pd.NA

    # Normalize non-standard range indicators and remove currency indicators
    normalized_benefit = re.sub(r'[to]+', '-', benefit)
    normalized_benefit = re.sub(r'NZD|\$', '', normalized_benefit)

    # Adjust regex to capture numerical values and remove commas and spaces
    matches = re.findall(r'(\d{1,3}(?:[,\s]\d{3})*\.?\d*)(k)?', normalized_benefit)
    numbers = [float(num.replace(',', '').replace(' ', '')) * 1000 if k else float(num.replace(',', '').replace(' ', '')) for num, k in matches]

    # Determine if it's an annual salary based on the magnitude of the number
    annual_salary_threshold = 1000  # Adjust this threshold as needed
    is_annual_salary = any(num > annual_salary_threshold for num in numbers)

    if '-' in normalized_benefit:
        # When the benefit contains a range
        lower, upper = numbers[0], numbers[1] if len(numbers) == 2 else numbers[0]
        if is_annual_salary:
            # Convert annual salary to hourly rate
            lower /= 2080
            upper /= 2080
        return lower, upper

    # When the benefit is a single value
    lower = numbers[0]
    if is_annual_salary:
        # Convert annual salary to hourly rate
        lower /= 2080
    return lower, pd.NA

# Applying the function
df[['lower_salary', 'upper_salary']] = df.apply(lambda x: pd.Series(separate_salaries(x['benefit'])), axis=1)


In [15]:
df.head(50)

,position,url,company,location,elapsed_time_since_posted,classification,area,benefit,sub_classification,lower_salary,upper_salary
0,Administrator,https://www.seek.co.nz/job/50582301?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,Bay of Plenty,<NA>,Administration & Office Support,Tauranga,<NA>,Office Management,<NA>,<NA>
1,Receptionist,https://www.seek.co.nz/job/50620889?type=promoted#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Avenues Orthodontics,Bay of Plenty,<NA>,Administration & Office Support,Tauranga,<NA>,Receptionists,<NA>,<NA>
2,Prosecutions Support Officer,https://www.seek.co.nz/job/50622169?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,New Zealand Police,Auckland,96,Administration & Office Support,<NA>,<NA>,Other,<NA>,<NA>
3,Early Childhood Centre Administrator,https://www.seek.co.nz/job/50639620?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Kew Pacific Island Early Learning Centre,Southland,1,Administration & Office Support,Invercargill,<NA>,Administrative Assistants,<NA>,<NA>
4,Business Support Administrator,https://www.seek.co.nz/job/50622432?type=standout#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Private Advertiser,Canterbury,96,Administration & Office Support,Christchurch,<NA>,Client & Sales Administration,<NA>,<NA>
5,Support Officer,https://www.seek.co.nz/job/50640393?type=standout#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Ministry for Primary Industries,Northland,6480,Administration & Office Support,Whangarei,<NA>,Administrative Assistants,<NA>,<NA>
6,Support Officer,https://www.seek.co.nz/job/50615674?type=standout#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,"Ministry of Business, Innovation and Employment",Wellington,120,Administration & Office Support,Wellington Central,<NA>,Other,<NA>,<NA>
7,Office Administrator,https://www.seek.co.nz/job/50640166?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Hepburn Electrical Ltd,Bay of Plenty,5040,Administration & Office Support,Rotorua,<NA>,Administrative Assistants,<NA>,<NA>
8,Office Administrator,https://www.seek.co.nz/job/50639248?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,Webster Holland Ltd,Bay of Plenty,2,Administration & Office Support,Tauranga,<NA>,Administrative Assistants,<NA>,<NA>
9,Administration Officer,https://www.seek.co.nz/job/50629393?type=standard#searchRequestToken=feee129e-c80f-4f79-ac5f-98ddb6d6c22b,New Zealand Police,Canterbury,72,Administration & Office Support,<NA>,<NA>,Other,<NA>,<NA>


In [16]:
df.isnull().sum()

position                        0
url                             0
company                         0
location                        0
elapsed_time_since_posted      20
classification                603
area                          541
benefit                      2105
sub_classification            603
lower_salary                 2612
upper_salary                 2618
dtype: int64

In [17]:
df.tail(50)

,position,url,company,location,elapsed_time_since_posted,classification,area,benefit,sub_classification,lower_salary,upper_salary
2658,"Resource Coordinator - Construction (Plant, Material, Supplies)",https://www.seek.co.nz/job/50487373?type=standout#searchRequestToken=e5acda48-6658-4bfc-a369-f2b6e87af958,Piritahi,Auckland,648,Administration & Office Support,Auckland Central,<NA>,Administrative Assistants,<NA>,<NA>
2659,Customer Service And Sales Support,https://www.seek.co.nz/job/50490247?type=standout#searchRequestToken=e5acda48-6658-4bfc-a369-f2b6e87af958,Wellfit-Wstar,Auckland,648,Administration & Office Support,Auckland Central,<NA>,Client & Sales Administration,<NA>,<NA>
2660,Experienced Administrator,https://www.seek.co.nz/job/50490876?type=standard#searchRequestToken=e5acda48-6658-4bfc-a369-f2b6e87af958,Adecco Office,Canterbury,648,Administration & Office Support,Christchurch,<NA>,Administrative Assistants,<NA>,<NA>
2661,Category Manager,https://www.seek.co.nz/job/50524687?type=standout#searchRequestToken=e5acda48-6658-4bfc-a369-f2b6e87af958,Bidfood,Auckland,480,Administration & Office Support,Auckland Central,<NA>,Other,<NA>,<NA>
2662,Administrator - Temps,https://www.seek.co.nz/job/50478359?type=standout#searchRequestToken=e5acda48-6658-4bfc-a369-f2b6e87af958,Recruitment Studio,Auckland,672,<NA>,Auckland Central,Competitive hourly rate depending on experience,<NA>,<NA>,<NA>
2663,Temp Receptionists/Administrators - Ad Agencies/Music Companies,https://www.seek.co.nz/job/50488787?type=standard#searchRequestToken=e5acda48-6658-4bfc-a369-f2b6e87af958,Metro Recruitment Ltd,Auckland,648,Administration & Office Support,Auckland Central,<NA>,Receptionists,<NA>,<NA>
2664,Graduates With Office Administration Experience!,https://www.seek.co.nz/job/50481639?type=standout#searchRequestToken=e5acda48-6658-4bfc-a369-f2b6e87af958,GBL Personnel,Wellington,672,Administration & Office Support,<NA>,<NA>,Administrative Assistants,<NA>,<NA>
2665,On-Site Manager,https://www.seek.co.nz/job/50481773?type=standard#searchRequestToken=e5acda48-6658-4bfc-a369-f2b6e87af958,Stonewood Group Ltd,Wellington,672,Administration & Office Support,Wellington Central,<NA>,Receptionists,<NA>,<NA>
2666,Business Professional - Temporary Opportunities,https://www.seek.co.nz/job/50482020?type=standout#searchRequestToken=e5acda48-6658-4bfc-a369-f2b6e87af958,Enterprise Recruitment,Canterbury,672,Administration & Office Support,Christchurch,<NA>,Office Management,<NA>,<NA>
2667,Seeking Executive Support Candidates,https://www.seek.co.nz/job/50504568?type=standout#searchRequestToken=e5acda48-6658-4bfc-a369-f2b6e87af958,GBL Personnel,Wellington,600,Administration & Office Support,<NA>,<NA>,Administrative Assistants,<NA>,<NA>
